In [2]:
import xosrm
import pandas as pd
import pprint

In [3]:
def open_excel_file(file_path):
    '''Open Excel file and create DataFrame
    input: filepath
    output: DataFrame'''
    data = pd.read_excel(file_path)
    return data

def schedule(df):

    week = {}
    week['mon'] = df[df['Дни недели']==1].to_dict('records')
    week['tue'] = df[df['Дни недели']==2].to_dict('records')
    week['wed'] = df[df['Дни недели']==3].to_dict('records')
    week['thu'] = df[df['Дни недели']==4].to_dict('records')
    week['fri'] = df[df['Дни недели']==5].to_dict('records')
    return week


def schedule_odd_even(df):

    week = {}
    week['чет'] = {}
    week['не чет'] = {}
    
    even = df[df['№п/п четная нед.'].notnull()]
    week['чет']['ПН'] = even[even['Дни недели']==1].to_dict('records')
    week['чет']['ВТ'] = even[even['Дни недели']==2].to_dict('records')
    week['чет']['СР'] = even[even['Дни недели']==3].to_dict('records')
    week['чет']['ЧТ'] = even[even['Дни недели']==4].to_dict('records')
    week['чет']['ПТ'] = even[even['Дни недели']==5].to_dict('records')
    
    odd = df[df['№п/п не четная нед.'].notnull()]        
    week['не чет']['ПН'] = odd[odd['Дни недели']==1].to_dict('records')
    week['не чет']['ВТ'] = odd[odd['Дни недели']==2].to_dict('records')
    week['не чет']['СР'] = odd[odd['Дни недели']==3].to_dict('records')
    week['не чет']['ЧТ'] = odd[odd['Дни недели']==4].to_dict('records')
    week['не чет']['ПТ'] = odd[odd['Дни недели']==5].to_dict('records')
    return week

In [7]:
schedule_data = open_excel_file(
    file_path='C:\\Users\\andre\\OneDrive\\Рабочий стол\\sav\\Odessa\\Аптечный ТП Copy.xlsx')
# Dict of coordinate for each day
full_dict = schedule_odd_even(schedule_data)
full_dict

{'чет': {'ПН': [{'Внешний ID ТТ': 8000015163380,
    'Клиент': 'Новий для КПК в м. Одеса',
    'Адрес': 'Одеська обл., м.Одеса, вул. Аеропортівська 4 .',
    'Интервал повторений': 1,
    'Дни недели': 1,
    '№п/п не четная нед.': 0.0,
    '№п/п четная нед.': 0.0,
    'Долгота': 30.6508,
    'Широта': 46.4289},
   {'Внешний ID ТТ': 8000017079473,
    'Клиент': 'Київське',
    'Адрес': 'Одеська обл., м.Одеса, вул. Асташкіна 29 корпус 2, приміщення 103-Н магазин',
    'Интервал повторений': 1,
    'Дни недели': 1,
    '№п/п не четная нед.': 1.0,
    '№п/п четная нед.': 1.0,
    'Долгота': 30.723,
    'Широта': 46.4788},
   {'Внешний ID ТТ': 8000016371065,
    'Клиент': 'Київське',
    'Адрес': 'Одеська обл., м.Одеса, вул. Михайла Грушевського 39/4 .',
    'Интервал повторений': 1,
    'Дни недели': 1,
    '№п/п не четная нед.': 1.0,
    '№п/п четная нед.': 1.0,
    'Долгота': 30.6884,
    'Широта': 46.4806},
   {'Внешний ID ТТ': 8000018554061,
    'Клиент': 'Київське',
    'Адрес': 'Оде

In [8]:
def calc_trips(dict_of_coords_by_day):
    ''' Calculate TRIP_ROUTE
    Calculate distances and durations for each day of the week
    Output: dict of geometries for each day
    '''    
    # Dict of geometries of route by day
    result_dict = full_dict.copy()
    # Create dict of distances
    dist_dict = {}
    for key0, value0 in result_dict.items():
        dist_dict[key0] = {}
        for key2, value2 in value0.items():
            print(key2)
            coords = []
            for point in value2:
                coords.append([point['Долгота'], point['Широта']])
            result = xosrm.trip(coords, source='first', roundtrip=True,
                                output='full', overview="full", geometry="geojson")
            distance = result['trips'][0]['distance']/1000
            dist_dict[key0][key2] = distance

            result2 = xosrm.trip(coords, source='first', roundtrip=True,
                                output='only_index', overview="full", geometry="geojson")
            for wp in result2:
                print(wp['waypoint'], end=' ')

#                 for point in value2:
                    
                    

            for index, point in enumerate(value2):
                if key0 == 'чет':
                    point.update({'№п/п четная нед.': result2[index]['waypoint']})
                    point.update({'№п/п не четная нед.': None})
                if key0 == 'не чет':
                    point.update({'№п/п не четная нед.': result2[index]['waypoint']})
                    point.update({'№п/п четная нед.': None})
                    
    return dist_dict, result_dict

data = calc_trips(full_dict)
display(data[0])
display(data[0])


ПН
0 5 2 1 3 11 4 18 16 20 15 6 17 14 8 7 12 21 13 19 10 9 ВТ
0 11 13 7 5 16 18 17 12 8 15 14 19 9 6 4 10 3 2 1 СР
0 12 6 20 19 16 18 22 11 10 3 2 1 5 7 8 9 13 14 21 15 23 17 4 ЧТ
0 1 ПТ
0 3 2 1 4 ПН
0 4 1 36 6 31 5 28 22 29 26 20 25 27 18 13 34 10 14 33 11 19 7 12 21 23 24 35 8 9 3 2 17 16 15 30 32 ВТ
0 4 2 6 9 1 13 11 8 5 16 14 12 7 10 3 15 СР
0 22 9 4 3 5 2 8 26 25 33 35 36 21 23 6 31 16 20 37 7 1 13 10 28 29 17 18 19 27 30 24 32 14 15 12 11 34 ЧТ
0 1 3 6 7 8 9 5 11 2 10 4 ПТ
0 5 6 1 4 11 2 3 10 9 7 8 

{'чет': {'ПН': 45.5464,
  'ВТ': 70.0253,
  'СР': 81.9529,
  'ЧТ': 16.586299999999998,
  'ПТ': 29.4997},
 'не чет': {'ПН': 48.0489,
  'ВТ': 61.9958,
  'СР': 64.9587,
  'ЧТ': 33.1125,
  'ПТ': 40.071400000000004}}

{'чет': {'ПН': 45.5464,
  'ВТ': 70.0253,
  'СР': 81.9529,
  'ЧТ': 16.586299999999998,
  'ПТ': 29.4997},
 'не чет': {'ПН': 48.0489,
  'ВТ': 61.9958,
  'СР': 64.9587,
  'ЧТ': 33.1125,
  'ПТ': 40.071400000000004}}

In [28]:

df = pd.DataFrame.from_dict(data[1]['чет'], orient='columns')
df


ValueError: arrays must all be same length

In [9]:

writer = pd.ExcelWriter('schedule.xlsx', engine = 'xlsxwriter')

for key0, value0 in data[1].items():
    for key2, value2 in value0.items():
        df = pd.DataFrame.from_dict(value2, orient='columns')
        print(key2)
        name = '-'.join([key0, key2])
        df.to_excel(writer, sheet_name = name)

writer.save()
writer.close()

ПН
ВТ
СР
ЧТ
ПТ
ПН
ВТ
СР
ЧТ
ПТ


In [83]:
import folium
from folium import plugins

# for key0, value0 in data[1].items():
#     for key, value in value0.items():
#         for key2, value2 in value.items():
dataset1 = data[1]['first_part']['чет']['5-пт']
dataset2 = data[1]['first_part']['не чет']['5-пт']
coords1 = []
coords2 = []
for point in dataset1:
    coords1.append([point['Широта'], point['Долгота']])
for point in dataset2:
    coords2.append([point['Широта'], point['Долгота']])

m = folium.Map( location=[46.4289,30.650], zoom_start=5 )


plugins.Fullscreen(
position='topright',
title='FullScreen',
title_cancel='Exit me',
force_separate_button=True).add_to(m)

for coord in coords1:
    folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=8 ).add_to(m)
for coord in coords2:
    folium.CircleMarker( location=[ coord[0], coord[1] ], radius=10, color='red',
    fill=True,
    fill_color='#3186cc').add_to(m)

m
                    
                
                

In [20]:
def calc_dist(full_dict):
    ''' Calculate distances for each day of the week
    Output: dict of distances
    '''
    result_dict = full_dict.copy()
    # Create dict of distances
    dist_dict = {}
    for key0, value0 in result_dict.items():
        dist_dict[key0] = {}
        for key, value in value0.items():
            dist_dict[key0][key]= {}
            for key2, value2 in value.items():
                coords = []
                for point in value2:
                    coords.append([point['Долгота'], point['Широта']])
                # Office Oddessa
                result = xosrm.simple_route([30.6508, 46.4289], [30.6508, 46.4289], coords,
                                    output='full', overview="full", geometry="geojson")

                # Add distances to dict
                distance = result['routes'][0]['distance']/1000
                dist_dict[key0][key][key2] = distance
    return dist_dict

data = calc_dist(data[1])
display(df_first(data))
display(df_second(data))

,1-monday,2-tuesday,3-wednesday,4-thursday,5-friday
even_part_of_coords,37.5967,127.4752,34.4968,259.5765,163.9633
odd_part_of_coords,32.0182,209.8830,33.2799,191.9509,154.5969


,1-monday,2-tuesday,3-wednesday,4-thursday,5-friday
even_part_of_coords,37.5967,127.4752,34.4968,219.5573,150.1969
odd_part_of_coords,32.7166,209.8830,33.2799,188.7929,154.4203
